## Population Resource Consumption

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../../az-function/")


import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

Much of the Population growth is handled in scripts inside the Azure Function

In [2]:
# mapping to the modules that make the app

from app.connectors import cmdb_graph
from app.functions import consumption
from app.functions import growth
from app.objects import time

c = cmdb_graph.CosmosdbClient()


executing local windows deployment
loading syllables from  c:\Users\willi\repos\exoplanets\notebooks\People


In [3]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"app/configurations/popgrowthconfig.yaml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'

Time is updated by the `time` function. It is always running.

In [4]:
t = time.Time(c)
t.get_current_UTU()
params['currentTime'] = t.params['currentTime']
t

< time at UTU:55881 >

In [5]:
params

{'pop_health_requirement': 0.7,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 55881}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

# Calculate Consumption

the first step is to get a list of all of the people who will consume. 

In [10]:
consuming_pops_query = "g.V().hasLabel('pop').as('pop').out('isOf').as('species').path().by('objid').by(values('consumes','effuses').fold())"
c.run_query(consuming_pops_query)
consuming_pops_res = [p['objects'] for p in c.res]
consuming_pops_res


[['2775406017619', ['organics', 'organic waste,plastics']],
 ['7646745695043', ['organics', 'organic waste,plastics']],
 ['1561022813030', ['organics', 'organic waste,plastics']],
 ['4084988156125', ['organics', 'organic waste,plastics']],
 ['3159775761251', ['organics', 'organic waste,plastics']],
 ['4377972941053', ['organics', 'organic waste,plastics']],
 ['6359970696314', ['organics', 'organic waste,plastics']]]

In [12]:
consuming_pops = []
for i in consuming_pops_res:
    a = {'objid': i[0], 'consumes': i[1][0].split(), 'effuses': i[1][1].split(',')}
    consuming_pops.append(a)

consuming_pops

[{'objid': '2775406017619',
  'consumes': 'organics',
  'effuses': ['organic waste', 'plastics']},
 {'objid': '7646745695043',
  'consumes': 'organics',
  'effuses': ['organic waste', 'plastics']},
 {'objid': '1561022813030',
  'consumes': 'organics',
  'effuses': ['organic waste', 'plastics']},
 {'objid': '4084988156125',
  'consumes': 'organics',
  'effuses': ['organic waste', 'plastics']},
 {'objid': '3159775761251',
  'consumes': 'organics',
  'effuses': ['organic waste', 'plastics']},
 {'objid': '4377972941053',
  'consumes': 'organics',
  'effuses': ['organic waste', 'plastics']},
 {'objid': '6359970696314',
  'consumes': 'organics',
  'effuses': ['organic waste', 'plastics']}]

Then we will create an action event for each pop

In [14]:
message = consumption.get_consumption_message(consuming_pops[0])
message

{'agent': {'objid': '2775406017619',
  'consumes': 'organics',
  'effuses': ['organic waste', 'plastics']},
 'action': 'consume'}

The message is then sent to `resolveActionEvents` to be proccessed in parralel. 

## Consuming



In [21]:
outgoing_messages = []
if message.get('action')=="consume":
    for resource in message['agent']['consumes']:
        # outgoing_messages += consumption.reduce_location_or_faction_resource(c,t,message,resource)
        print(f"EXOADMIN:       -------And with that processed CONSUMPTION: {message['agent']} at UTU:{t}")


outgoing_messages

EXOADMIN:       -------And with that processed CONSUMPTION: {'objid': '2775406017619', 'consumes': 'organics', 'effuses': ['organic waste', 'plastics']} at UTU:< time at UTU:55881 >
EXOADMIN:       -------And with that processed CONSUMPTION: {'objid': '2775406017619', 'consumes': 'organics', 'effuses': ['organic waste', 'plastics']} at UTU:< time at UTU:55881 >
EXOADMIN:       -------And with that processed CONSUMPTION: {'objid': '2775406017619', 'consumes': 'organics', 'effuses': ['organic waste', 'plastics']} at UTU:< time at UTU:55881 >
EXOADMIN:       -------And with that processed CONSUMPTION: {'objid': '2775406017619', 'consumes': 'organics', 'effuses': ['organic waste', 'plastics']} at UTU:< time at UTU:55881 >
EXOADMIN:       -------And with that processed CONSUMPTION: {'objid': '2775406017619', 'consumes': 'organics', 'effuses': ['organic waste', 'plastics']} at UTU:< time at UTU:55881 >
EXOADMIN:       -------And with that processed CONSUMPTION: {'objid': '2775406017619', 'co

In [26]:
resource = message['agent']['consumes'][0]
objid = message['agent']['objid']

faction_resource_query = f"""
g.V().has('objid','{objid}').out('isIn').out('has').valuemap()
"""
c.run_query(resource_query)
resource_res = c.clean_nodes(c.res)
resource_res

[]

# Renewing Resources

In [13]:
renewing_resources_query =f"""
   g.V().has('label','resource').has('replenish_rate').valuemap()
"""

In [14]:
c.run_query(renewing_resources_query)
renewing_resources = c.clean_nodes(c.res)
renewing_resources

[{'replenish_rate': 10,
  'volume': 159,
  'name': 'Organic',
  'objid': '4182507921316',
  'max_volume': 895,
  'description': 'bilogical material that can be consumed by pops',
  'username': 'manderson',
  'objtype': 'resource',
  'id': '4182507921316'},
 {'replenish_rate': 10,
  'volume': 9,
  'name': 'organics',
  'objid': '9706157180627',
  'max_volume': 1095,
  'description': 'bilogical material that can be consumed by pops',
  'username': 'Billmanserver',
  'objtype': 'resource',
  'id': '9706157180627'},
 {'replenish_rate': 10,
  'volume': 525,
  'name': 'organics',
  'objid': '0058029712212',
  'max_volume': 1051,
  'description': 'bilogical material that can be consumed by pops',
  'username': 'BillmanLocal2',
  'objtype': 'resource',
  'id': '0058029712212'},
 {'replenish_rate': 10,
  'name': 'organics',
  'objid': '7450760547046',
  'volume': 850,
  'max_volume': 850,
  'description': 'bilogical material that can be consumed by pops',
  'username': 'william.jeffrey.harding@

In [12]:
renewal_messages = []
for resource in renewing_resources:
    if resource['volume'] <= resource['max_volume']:
        renewal_message = growth.get_renewal_message(resource)
        renewal_messages.append(renewal_message)

renewal_messages

[{'agent': {'replenish_rate': 10,
   'volume': 159,
   'name': 'Organic',
   'objid': '4182507921316',
   'max_volume': 895,
   'description': 'bilogical material that can be consumed by pops',
   'username': 'manderson',
   'objtype': 'resource',
   'id': '4182507921316'},
  'action': 'renew'},
 {'agent': {'replenish_rate': 10,
   'volume': 9,
   'name': 'organics',
   'objid': '9706157180627',
   'max_volume': 1095,
   'description': 'bilogical material that can be consumed by pops',
   'username': 'Billmanserver',
   'objtype': 'resource',
   'id': '9706157180627'},
  'action': 'renew'},
 {'agent': {'replenish_rate': 10,
   'volume': 525,
   'name': 'organics',
   'objid': '0058029712212',
   'max_volume': 1051,
   'description': 'bilogical material that can be consumed by pops',
   'username': 'BillmanLocal2',
   'objtype': 'resource',
   'id': '0058029712212'},
  'action': 'renew'},
 {'agent': {'replenish_rate': 10,
   'name': 'organics',
   'objid': '7450760547046',
   'volume': 

## Starving


In [4]:
example_location = "6135767546801"

starving_pop_query =f"""
   g.V().has('objid','{example_location}').in('inhabits').valuemap()
"""

c.run_query(starving_pop_query)
starving_pops = c.clean_nodes(c.res)
starving_pops

[{'isIdle': 'true',
  'health': 0.8,
  'wealth': 0.4685,
  'name': 'Bergcas Trogo',
  'objid': '2792627071778',
  'conformity': 0.01,
  'literacy': 0.45,
  'aggression': 0.55,
  'constitution': 0.824,
  'isIn': '9324041486564',
  'industry': 0.687,
  'factionLoyalty': 0.079,
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'pop',
  'id': '2792627071778'},
 {'name': 'Ramvinorth Tre',
  'objid': '9972068386304',
  'conformity': 0.172,
  'literacy': 0.304,
  'aggression': 0.6,
  'constitution': 0.675,
  'health': 0.7,
  'isIn': '6306635719673',
  'industry': 0.6375,
  'wealth': 0.4708,
  'factionLoyalty': 0.067,
  'isIdle': 'true',
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'pop',
  'id': '9972068386304'},
 {'isIdle': 'true',
  'health': 0.8,
  'wealth': 0.4532,
  'name': 'Bergcas Nisrio',
  'objid': '3458241253622',
  'conformity': 0.136,
  'literacy': 0.459,
  'aggression': 0.41,
  'constitution': 0.885,
  'isIn': '9324041486564',
  'industry': 

In [14]:
starving_action = {
    "type": "starve",
    "label": "action",
    "applies_to": "pop",
    "effort": 0,
    "augments_self_properties": {"health": -7},
    "comment": "Populations that don't have enough resources will loose health until it reachees zero"
}

In [15]:
starving_job = {
        'created_at':t.params['currentTime'],
        'objid':f"starving_async_{t.params['currentTime']}",
}
starving_job

{'created_at': 21325, 'objid': 'starving_async_21325'}

In [16]:
pop = starving_pops[0]

starving_action_obj = time.Action(c,{'agent':pop,'action':starving_action,'job':starving_job})
starving_action_obj

< (Bergcas Trogo: 2792627071778) -None:None-> (starve) >

In [17]:
starving_action_obj.get_action_message()

{'agent': {'isIdle': 'true',
  'health': 0.8,
  'wealth': 0.4685,
  'name': 'Bergcas Trogo',
  'objid': '2792627071778',
  'conformity': 0.01,
  'literacy': 0.45,
  'aggression': 0.55,
  'constitution': 0.824,
  'isIn': '9324041486564',
  'industry': 0.687,
  'factionLoyalty': 0.079,
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'pop',
  'id': '2792627071778'},
 'action': {'type': 'starve',
  'label': 'action',
  'applies_to': 'pop',
  'effort': 0,
  'augments_self_properties': {'health': -7},
  'comment': "Populations that don't have enough resources will loose health until it reachees zero"},
 'job': {'created_at': 21325, 'objid': 'starving_async_21325'}}